# Mapping Crime Categories to Crime Description

[data.police](https://data.police.uk/) data contains crime categories data but this only splits crimes into 15 different categories. However [police recorded crime](https://www.gov.uk/government/statistics/police-recorded-crime-open-data-tables) captures Crime Description that captures 132 different descriptions.

The key work of this notebook is to try and match both up into a dictionary of values for use in the crime_sim_toolkit.

In [1]:
import pandas as pd
import glob
import pprint

In [2]:
pp = pprint.PrettyPrinter()

In [3]:
# import police recorded crime data 201718
prcdata = pd.read_csv(glob.glob('/home/alex/Downloads/*201718.csv')[0])

In [4]:
filel = glob.glob('/home/alex/Code/python/crime_sim_toolkit/crime_sim_toolkit/tests/testing_data/*counts.csv')

datadotpolice = pd.read_csv(filel[0])

In [5]:
datadotpolice.columns = datadotpolice.columns.str.replace(' ','_')

prcdata.columns = prcdata.columns.str.replace(' ','_')

In [6]:
mappings = pd.read_csv('https://data.police.uk/static/files/police-uk-category-mappings.csv', encoding='latin')

In [7]:
mappings.columns = mappings.columns.str.replace(' ','_')

In [8]:
pp.pprint(mappings['Police.uk_Category'].unique().tolist())

['Violence and Sexual Offences',
 'Possession of Weapons',
 'Public Order',
 'Other Crime',
 'Burglary',
 'Robbery',
 'Other Theft',
 'Vehicle Crime',
 'Theft from the Person',
 'Bicycle Theft',
 'Shoplifting',
 'Criminal Damage and Arson',
 'Drugs']


In [9]:
pp.pprint(prcdata.Offence_Group.unique().tolist())

['Fraud offences',
 'Miscellaneous crimes',
 'Sexual offences',
 'Theft offences',
 'Criminal damage and arson',
 'Violence against the person',
 'Drug offences',
 'Possession of weapons offences',
 'Public order offences',
 'Robbery']


In [10]:
pp.pprint(prcdata.Offence_Subgroup.unique().tolist())

['Fraud: action fraud',
 'Miscellaneous crimes',
 'Other sexual offences',
 'Non-domestic burglary',
 'Domestic burglary',
 'Theft of a motor vehicle',
 'Arson',
 'Violence with injury',
 'Violence without injury',
 'Other theft offences',
 'Criminal damage',
 'Stalking and harassment',
 'Homicide',
 'Vehicle interference',
 'Possession of drugs',
 'Possession of weapons offences',
 'Public order offences',
 'Rape offences',
 'Robbery of business property',
 'Robbery of personal property',
 'Shoplifting',
 'Theft from the person',
 'Theft from a vehicle',
 'Bicycle theft',
 'Trafficking of drugs',
 'Fraud: CIFAS',
 'Fraud: Financial Fraud Action UK',
 'Fraud: UK Finance']


In [11]:
pp.pprint(prcdata.Offence_Description.unique().tolist())

['Fraud offences recorded by Action Fraud',
 'Absconding from lawful custody',
 'Abuse of children through sexual exploitation',
 'Abuse of position of trust of a sexual nature',
 'Aggravated Burglary Business and Community',
 'Aggravated burglary in a building other than a dwelling(outcome only)',
 'Aggravated burglary in a dwelling (outcome only)',
 'Aggravated Burglary Residential',
 'Aggravated vehicle taking',
 'Aiding suicide',
 'Arson endangering life',
 'Arson not endangering life',
 'Assault with injury',
 'Assault with injury on a constable',
 'Assault with intent to cause serious harm',
 'Assault without injury',
 'Assault without injury on a constable',
 'Attempted Burglary Business and Community',
 'Attempted burglary in a building other than a dwelling (outcome only)',
 'Attempted burglary in a dwelling (outcome only)',
 'Attempted Burglary Residential',
 'Attempted distraction burglary in a dwelling (outcome only)',
 'Attempted Distraction Burglary Residential',
 'Attemp

### Approximate heuristics

To cluster these offense descriptions into the crime types I'll use the FAQs found on [police.uk/](https://www.police.uk/about-this-site/faqs/#what-do-the-crime-categories-mean) as a rought guideline.

In [12]:
groupedprc = prcdata.groupby(['Offence_Group','Offence_Subgroup'])['Offence_Description'].value_counts().reset_index('Offence_Subgroup')

groupedprc.columns = ['Offence_Subgroup','Counts']

groupedprc = groupedprc[['Offence_Subgroup']]

groupedprc.reset_index(['Offence_Group','Offence_Description'], inplace=True)

In [13]:
groupedprc

,Offence_Group,Offence_Description,Offence_Subgroup
0,Criminal damage and arson,Arson endangering life,Arson
1,Criminal damage and arson,Arson not endangering life,Arson
2,Criminal damage and arson,Criminal damage to a building other than a dwe...,Criminal damage
3,Criminal damage and arson,Criminal damage to a dwelling,Criminal damage
4,Criminal damage and arson,Criminal damage to a vehicle,Criminal damage
5,Criminal damage and arson,Other criminal damage,Criminal damage
6,Criminal damage and arson,Racially or religiously aggravated criminal da...,Criminal damage
7,Drug offences,Other drug offences,Possession of drugs
8,Drug offences,Possession of controlled drugs (Cannabis),Possession of drugs
9,Drug offences,Possession of controlled drugs (excl. Cannabis),Possession of drugs


In [14]:
mappings = pd.read_csv('https://data.police.uk/static/files/police-uk-category-mappings.csv', encoding='latin')

In [15]:
mappings.columns = mappings.columns.str.replace(' ','_')

In [16]:
mappings.head()

,Home_Office_Code,Sub_Class,Offence,Police.uk_Category
0,1/1,Homicide,Murder of persons aged 1 yr or over,Violence and Sexual Offences
1,1/1,Homicide,Genocide or crime against humanity,Violence and Sexual Offences
2,1/2,Homicide,Murder of persons under 1yr of age,Violence and Sexual Offences
3,2,Violence with injury,Attempted murder,Violence and Sexual Offences
4,2,Violence with injury,Attempted genocide or crime against humanity,Violence and Sexual Offences


In [17]:
mappings.Home_Office_Code.unique()

array(['1/1', '1/2', '2', ..., '825/18', '825/20', '825/24'], dtype=object)

In [18]:
groupedprc.Offence_Description.tolist()[0]

'Arson endangering life'

In [19]:
counts = 0

for des in groupedprc.Offence_Description.unique():
    
    if mappings.Offence.isin([des]).sum() > 0:
        
        print(des)

Arson endangering life
Absconding from lawful custody
Bigamy
Violent disorder
Sexual assault on a male child under 13
Blackmail
Making off without payment
Theft by an employee
Theft from automatic machine or meter
Aggravated vehicle taking
Harassment
Attempted murder
Causing death by aggravated vehicle taking
Kidnapping


In [20]:
counts = 0

for des in prcdata.Offence_Code.unique():
    
    if mappings.Home_Office_Code.isin([des]).sum() > 0:
        
        counts += 1

print(counts)

17


In [21]:
# clearly the crime code mapping does not match up sufficiently with the police recorded data

In [22]:
# therefore we'll look to annotate a list manually using the mappings as a guide

In [23]:
# instantiate an initial dataframe to work with
init_frame = groupedprc

In [24]:
pp.pprint(groupedprc.Offence_Group.unique().tolist())

['Criminal damage and arson',
 'Drug offences',
 'Fraud offences',
 'Miscellaneous crimes',
 'Possession of weapons offences',
 'Public order offences',
 'Robbery',
 'Sexual offences',
 'Theft offences',
 'Violence against the person']


In [25]:
vio_subframe = init_frame[init_frame.Offence_Group.isin(['Violence against the person','Sexual offences'])]

In [26]:
# yup visual inspection time and compare to mapping
vio_subframe

,Offence_Group,Offence_Description,Offence_Subgroup
51,Sexual offences,Abuse of children through sexual exploitation,Other sexual offences
52,Sexual offences,Abuse of position of trust of a sexual nature,Other sexual offences
53,Sexual offences,Causing sexual activity without consent,Other sexual offences
54,Sexual offences,Exposure and voyeurism,Other sexual offences
55,Sexual offences,Incest or familial sexual offences,Other sexual offences
56,Sexual offences,Other miscellaneous sexual offences,Other sexual offences
57,Sexual offences,Sexual activity etc with a person with a menta...,Other sexual offences
58,Sexual offences,Sexual activity involving a child under 13,Other sexual offences
59,Sexual offences,Sexual activity involving child under 16,Other sexual offences
60,Sexual offences,Sexual assault on a female aged 13 and over,Other sexual offences


In [27]:
PO_subframe = init_frame[init_frame.Offence_Group.isin(['Public order offences'])]

In [28]:
PO_subframe

,Offence_Group,Offence_Description,Offence_Subgroup
45,Public order offences,Other offences against the State or public order,Public order offences
46,Public order offences,"Public fear, alarm or distress",Public order offences
47,Public order offences,Racially or religiously aggravated public fear...,Public order offences
48,Public order offences,Violent disorder,Public order offences


In [29]:
rob_subframe = init_frame[init_frame.Offence_Group.isin(['Robbery'])]

In [30]:
rob_subframe

,Offence_Group,Offence_Description,Offence_Subgroup
49,Robbery,Robbery of business property,Robbery of business property
50,Robbery,Robbery of personal property,Robbery of personal property


In [31]:
burg_subframe = init_frame[init_frame.Offence_Group.isin(['Theft offences'])]

In [32]:
burg_subframe[burg_subframe.Offence_Description.str.lower().str.contains('burglary')]

,Offence_Group,Offence_Description,Offence_Subgroup
74,Theft offences,Aggravated Burglary Residential,Domestic burglary
75,Theft offences,Aggravated burglary in a dwelling (outcome only),Domestic burglary
76,Theft offences,Attempted Burglary Residential,Domestic burglary
77,Theft offences,Attempted Distraction Burglary Residential,Domestic burglary
78,Theft offences,Attempted burglary in a dwelling (outcome only),Domestic burglary
79,Theft offences,Attempted distraction burglary in a dwelling (...,Domestic burglary
80,Theft offences,Burglary Residential,Domestic burglary
81,Theft offences,Burglary in a dwelling(outcome only),Domestic burglary
82,Theft offences,Distraction Burglary Residential,Domestic burglary
83,Theft offences,Distraction burglary in a dwelling (outcome only),Domestic burglary


In [33]:
theft_subframe = burg_subframe[(~burg_subframe.Offence_Description.str.lower().str.contains('burglary')) & 
                              (burg_subframe.Offence_Subgroup != 'Other theft offences')]

theft_subframe

,Offence_Group,Offence_Description,Offence_Subgroup
73,Theft offences,Theft or unauthorised taking of a pedal cycle,Bicycle theft
98,Theft offences,Shoplifting,Shoplifting
99,Theft offences,Theft from vehicle,Theft from a vehicle
100,Theft offences,Theft from the person,Theft from the person
101,Theft offences,Aggravated vehicle taking,Theft of a motor vehicle
102,Theft offences,Theft or unauthorised taking of motor vehicle,Theft of a motor vehicle
103,Theft offences,Interfering with a motor vehicle,Vehicle interference


In [34]:
shoplift_frame = theft_subframe[theft_subframe.Offence_Subgroup.isin(['Shoplifting'])]

shoplift_frame

,Offence_Group,Offence_Description,Offence_Subgroup
98,Theft offences,Shoplifting,Shoplifting


In [35]:
perstheft_frame = theft_subframe[theft_subframe.Offence_Subgroup.isin(['Theft from the person'])]

perstheft_frame

,Offence_Group,Offence_Description,Offence_Subgroup
100,Theft offences,Theft from the person,Theft from the person


In [36]:
cycltheft = theft_subframe[theft_subframe.Offence_Subgroup.isin(['Bicycle theft'])]

cycltheft

,Offence_Group,Offence_Description,Offence_Subgroup
73,Theft offences,Theft or unauthorised taking of a pedal cycle,Bicycle theft


In [37]:
other_theft_frame = init_frame[init_frame.Offence_Subgroup.str.lower().str.contains('other theft')]

other_theft_frame

,Offence_Group,Offence_Description,Offence_Subgroup
90,Theft offences,Blackmail,Other theft offences
91,Theft offences,Dishonest use of electricity,Other theft offences
92,Theft offences,Making off without payment,Other theft offences
93,Theft offences,Other theft,Other theft offences
94,Theft offences,Theft by an employee,Other theft offences
95,Theft offences,Theft from automatic machine or meter,Other theft offences
96,Theft offences,Theft in a dwelling other than from an automat...,Other theft offences
97,Theft offences,Theft of mail,Other theft offences


In [38]:
part_veh_frame = init_frame[(init_frame.Offence_Description.str.lower().str.contains('vehic'))
                           & (init_frame.Offence_Subgroup.str.lower().str.contains('vehic'))]

part_veh_frame

,Offence_Group,Offence_Description,Offence_Subgroup
99,Theft offences,Theft from vehicle,Theft from a vehicle
101,Theft offences,Aggravated vehicle taking,Theft of a motor vehicle
102,Theft offences,Theft or unauthorised taking of motor vehicle,Theft of a motor vehicle
103,Theft offences,Interfering with a motor vehicle,Vehicle interference


In [39]:
weaposs_frame = init_frame[init_frame.Offence_Group.str.lower().str.contains('possession')]



In [40]:
drug_frame = init_frame[init_frame.Offence_Group.str.lower().str.contains('drug')]

drug_frame

,Offence_Group,Offence_Description,Offence_Subgroup
7,Drug offences,Other drug offences,Possession of drugs
8,Drug offences,Possession of controlled drugs (Cannabis),Possession of drugs
9,Drug offences,Possession of controlled drugs (excl. Cannabis),Possession of drugs
10,Drug offences,Trafficking in controlled drugs,Trafficking of drugs


In [41]:
crimdam_frame = init_frame[init_frame.Offence_Group.str.lower().str.contains('damage')]

crimdam_frame

,Offence_Group,Offence_Description,Offence_Subgroup
0,Criminal damage and arson,Arson endangering life,Arson
1,Criminal damage and arson,Arson not endangering life,Arson
2,Criminal damage and arson,Criminal damage to a building other than a dwe...,Criminal damage
3,Criminal damage and arson,Criminal damage to a dwelling,Criminal damage
4,Criminal damage and arson,Criminal damage to a vehicle,Criminal damage
5,Criminal damage and arson,Other criminal damage,Criminal damage
6,Criminal damage and arson,Racially or religiously aggravated criminal da...,Criminal damage


In [42]:
# after eyeballing these selections lets build some code that will 
# rebuild the dataframe with a new column matching the PoliceData categories

init_frame = groupedprc

pile_of_slices = []

for cat in mappings['Police.uk_Category'].unique().tolist():
    
    if cat == 'Violence and Sexual Offences':
        
        subframe = init_frame[init_frame.Offence_Group.isin(['Violence against the person',
                                                                 'Sexual offences'])].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Possession of Weapons':
        
        subframe = init_frame[init_frame.Offence_Group.str.lower().str.contains('possession')].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)

    elif cat == 'Public Order':
        
        subframe = init_frame[init_frame.Offence_Group.isin(['Public order offences'])].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Other Crime':
        
        pass
    
    elif cat == 'Burglary':
        
        subframe = init_frame[init_frame.Offence_Group.isin(['Theft offences'])].copy()
        
        subframe = subframe[subframe.Offence_Description.str.lower().str.contains('burglary')]
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Robbery':
        
        subframe = init_frame[init_frame.Offence_Subgroup.str.lower().str.contains('robbery')].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat =='Other Theft':
        
        subframe = init_frame[init_frame.Offence_Subgroup.str.lower().str.contains('other theft')].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)

        
    elif cat == 'Vehicle Crime':
        
        subframe = init_frame[(init_frame.Offence_Description.str.lower().str.contains('vehic'))
                           & (init_frame.Offence_Subgroup.str.lower().str.contains('vehic'))].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)

    
    elif cat == 'Theft from the Person':
        
        subframe = init_frame[init_frame.Offence_Subgroup.isin(['Theft from the person'])].copy()
        
        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Bicycle Theft':
        
        subframe = init_frame[init_frame.Offence_Subgroup.isin(['Bicycle theft'])].copy()

        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Shoplifting':
        
        subframe = init_frame[init_frame.Offence_Subgroup.isin(['Shoplifting'])].copy()

        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Criminal Damage and Arson':
        
        subframe = init_frame[init_frame.Offence_Group.str.lower().str.contains('damage')].copy()

        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
    
    elif cat == 'Drugs':
        
        subframe = init_frame[init_frame.Offence_Group.str.lower().str.contains('drug')].copy()

        subframe['Police.uk_Cat'] = cat
        
        pile_of_slices.append(subframe)
        
cated_slices = pd.concat(pile_of_slices)

# quick line for misc crime

misc_crime = init_frame[~init_frame.Offence_Description.isin(cated_slices.Offence_Description.tolist())].copy()

misc_crime['Police.uk_Cat'] = 'Other crime'

final_ref_frame = pd.concat([cated_slices, misc_crime])

final_ref_frame.reset_index(drop=True, inplace=True)

In [43]:
cated_slices.shape

(104, 4)

In [44]:
cated_slices[cated_slices.Offence_Description.str.lower().str.contains('person')]

,Offence_Group,Offence_Description,Offence_Subgroup,Police.uk_Cat
57,Sexual offences,Sexual activity etc with a person with a menta...,Other sexual offences,Violence and Sexual Offences
118,Violence against the person,Causing or allowing death of child or vulnerab...,Violence with injury,Violence and Sexual Offences
126,Violence against the person,Cruelty to children/young persons,Violence without injury,Violence and Sexual Offences
50,Robbery,Robbery of personal property,Robbery of personal property,Robbery
100,Theft offences,Theft from the person,Theft from the person,Theft from the Person


In [45]:
cated_slices[cated_slices.Offence_Description.duplicated()]

,Offence_Group,Offence_Description,Offence_Subgroup,Police.uk_Cat


In [46]:
cated_slices.Offence_Description.duplicated().sum()

0

In [47]:
final_ref_frame.shape

(132, 4)

In [48]:
final_ref_frame.Offence_Description.duplicated().sum()

0

In [49]:
final_ref_frame

,Offence_Group,Offence_Description,Offence_Subgroup,Police.uk_Cat
0,Sexual offences,Abuse of children through sexual exploitation,Other sexual offences,Violence and Sexual Offences
1,Sexual offences,Abuse of position of trust of a sexual nature,Other sexual offences,Violence and Sexual Offences
2,Sexual offences,Causing sexual activity without consent,Other sexual offences,Violence and Sexual Offences
3,Sexual offences,Exposure and voyeurism,Other sexual offences,Violence and Sexual Offences
4,Sexual offences,Incest or familial sexual offences,Other sexual offences,Violence and Sexual Offences
5,Sexual offences,Other miscellaneous sexual offences,Other sexual offences,Violence and Sexual Offences
6,Sexual offences,Sexual activity etc with a person with a menta...,Other sexual offences,Violence and Sexual Offences
7,Sexual offences,Sexual activity involving a child under 13,Other sexual offences,Violence and Sexual Offences
8,Sexual offences,Sexual activity involving child under 16,Other sexual offences,Violence and Sexual Offences
9,Sexual offences,Sexual assault on a female aged 13 and over,Other sexual offences,Violence and Sexual Offences


In [50]:
final_ref_frame.to_csv('../crime_sim_toolkit/src/CrimeCatMappings.csv')

In [51]:
# for now we'll use 2017-2018 data as the reference for alloacting crime descriptions
# lets combine this mapping with this dataset

In [52]:
prcdata = pd.read_csv('../crime_sim_toolkit/src/prc-pfa-mar2013-onwards-tables201718.csv')

prcdata.columns = prcdata.columns.str.replace(' ','_')

In [53]:
prcdata.head()

,Financial_Year,Financial_Quarter,Force_Name,Offence_Description,Offence_Group,Offence_Subgroup,Offence_Code,Number_of_Offences
0,2017/18,1,Action Fraud,Fraud offences recorded by Action Fraud,Fraud offences,Fraud: action fraud,AF,66776
1,2017/18,1,Avon and Somerset,Absconding from lawful custody,Miscellaneous crimes,Miscellaneous crimes,80,2
2,2017/18,1,Avon and Somerset,Abuse of children through sexual exploitation,Sexual offences,Other sexual offences,71,6
3,2017/18,1,Avon and Somerset,Abuse of position of trust of a sexual nature,Sexual offences,Other sexual offences,73,1
4,2017/18,1,Avon and Somerset,Aggravated Burglary Business and Community,Theft offences,Non-domestic burglary,31A,2


In [54]:
final_ref_frame.head()

,Offence_Group,Offence_Description,Offence_Subgroup,Police.uk_Cat
0,Sexual offences,Abuse of children through sexual exploitation,Other sexual offences,Violence and Sexual Offences
1,Sexual offences,Abuse of position of trust of a sexual nature,Other sexual offences,Violence and Sexual Offences
2,Sexual offences,Causing sexual activity without consent,Other sexual offences,Violence and Sexual Offences
3,Sexual offences,Exposure and voyeurism,Other sexual offences,Violence and Sexual Offences
4,Sexual offences,Incest or familial sexual offences,Other sexual offences,Violence and Sexual Offences


In [55]:
prcdata['Policeuk_Cat'] = prcdata.Offence_Description.map(lambda x: final_ref_frame[final_ref_frame.Offence_Description.isin([x])]['Police.uk_Cat'].tolist()[0])

In [60]:
prcdata.to_csv('../crime_sim_toolkit/src/prc-pfa-201718_new.csv')

In [57]:
# building a simple reference table 

simple_table = prcdata.copy()

simple_table.columns

Index(['Financial_Year', 'Financial_Quarter', 'Force_Name',
       'Offence_Description', 'Offence_Group', 'Offence_Subgroup',
       'Offence_Code', 'Number_of_Offences', 'Policeuk_Cat'],
      dtype='object')

In [73]:
simple_table = simple_table[['Force_Name','Policeuk_Cat','Offence_Group',
              'Offence_Subgroup','Offence_Description','Number_of_Offences']]

In [91]:
descrip_totals = simple_table.groupby(['Force_Name','Policeuk_Cat','Offence_Description'])\
.agg({'Number_of_Offences' : 'sum'})

In [79]:
simple_perc_table = descrip_totals.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index(['Force_Name','Policeuk_Cat','Offence_Description'])

simple_perc_table.columns = ['Force_Name','Policeuk_Cat','Offence_Description','Percentage_of_offences']

simple_perc_table.head()

,Force_Name,Policeuk_Cat,Offence_Description,Percentage_of_offences
0,Action Fraud,Other crime,Fraud offences recorded by Action Fraud,100.000000
1,Avon and Somerset,Bicycle Theft,Theft or unauthorised taking of a pedal cycle,2.458039
2,Avon and Somerset,Burglary,Aggravated Burglary Business and Community,0.004997
3,Avon and Somerset,Burglary,Aggravated Burglary Residential,0.086385
4,Avon and Somerset,Burglary,Aggravated burglary in a building other than a...,0.000000


In [92]:
simple_table['Percentage_of_offences'] = simple_perc_table['Percentage_of_offences']

In [93]:
simple_table.head()

,Force_Name,Policeuk_Cat,Offence_Group,Offence_Subgroup,Offence_Description,Number_of_Offences,Percentage_of_offences
0,Action Fraud,Other crime,Fraud offences,Fraud: action fraud,Fraud offences recorded by Action Fraud,66776,100.000000
1,Avon and Somerset,Other crime,Miscellaneous crimes,Miscellaneous crimes,Absconding from lawful custody,2,2.458039
2,Avon and Somerset,Violence and Sexual Offences,Sexual offences,Other sexual offences,Abuse of children through sexual exploitation,6,0.004997
3,Avon and Somerset,Violence and Sexual Offences,Sexual offences,Other sexual offences,Abuse of position of trust of a sexual nature,1,0.086385
4,Avon and Somerset,Burglary,Theft offences,Non-domestic burglary,Aggravated Burglary Business and Community,2,0.000000


In [94]:
simple_table.to_csv('../crime_sim_toolkit/src/simple_policeuk_perc_offence.csv')